In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Neurona de McCulloch-Pitts (MP)

class MPNeuron:
    def __init__(self):
        self.threshold = None

    def model(self, x):
        # (1) si la suma de entradas >= umbral
        return (sum(x) >= self.threshold)

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(int(result))
        return np.array(Y)

    def fit(self, X, Y):
        # Aprendizaje por Fuerza Bruta
        accuracy = {}
        max_threshold = X.shape[1]
        
        print(f"Entrenando... Probando umbrales de 0 a {max_threshold}")
        
        for th in range(max_threshold + 1):
            self.threshold = th
            Y_pred = self.predict(X)
            accuracy[th] = accuracy_score(Y_pred, Y)
        
        # umbral con mejor accuracy
        self.threshold = max(accuracy, key=accuracy.get)
        print(f"Mejor umbral encontrado: {self.threshold} con Accuracy: {accuracy[self.threshold]:.4f}")

Breast Cancer con Split 55/45

In [2]:
# Modificación del Split (Breast Cancer)
# split a 55% Train / 45% Test

print("\n-DATASET BREAST CANCER (Split 55/45)-")
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data
y = data.target

# Binarización (Requisito de MP Neuron)
X_df = pd.DataFrame(X)
X_bin = X_df.apply(lambda x: pd.cut(x, bins=2, labels=[1, 0]), axis=0).values

# SPLIT 55% Train, 45% Test
X_train, X_test, y_train, y_test = train_test_split(
    X_bin, y, 
    test_size=0.45, 
    train_size=0.55, 
    stratify=y, 
    random_state=42
)

print(f"Datos de Entrenamiento: {X_train.shape[0]} muestras (55%)")
print(f"Datos de Prueba: {X_test.shape[0]} muestras (45%)")

# Entrenar y Evaluar
mp_neuron = MPNeuron()
mp_neuron.fit(X_train, y_train)

y_pred_test = mp_neuron.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Accuracy en Test (Split 55/45): {acc:.4f}")


-DATASET BREAST CANCER (Split 55/45)-
Datos de Entrenamiento: 312 muestras (55%)
Datos de Prueba: 257 muestras (45%)
Entrenando... Probando umbrales de 0 a 30
Mejor umbral encontrado: 28 con Accuracy: 0.8494
Accuracy en Test (Split 55/45): 0.8521


NSL-KDD Procesado

In [3]:
#IMPLEMENTACIÓN CON NSL-KDD
# Instrucción: Procesar y usar NSL-KDD en la neurona MP

print("\n" + "="*50)
print("-PROCESAMIENTO Y USO DE NSL-KDD-")
print("="*50)

try:
    df_train = pd.read_csv('kdd_train.csv')
    df_test = pd.read_csv('kdd_test.csv')
    print("Datasets cargados correctamente.")
except:
    print("ERROR: Sube los archivos kdd_train.csv y kdd_test.csv al entorno.")

# Preprocesamiento: Binarización del Objetivo (Target)
# 'normal' -> 0, Cualquier ataque -> 1
y_train_kdd = df_train['labels'].apply(lambda x: 0 if x == 'normal' else 1).values
y_test_kdd = df_test['labels'].apply(lambda x: 0 if x == 'normal' else 1).values

# se elimina la columna labels original para no usarla como feature
X_train_raw = df_train.drop('labels', axis=1)
X_test_raw = df_test.drop('labels', axis=1)

# Preprocesamiento: Codificación One-Hot para Categóricas
categorical_cols = ['protocol_type', 'service', 'flag']
X_train_enc = pd.get_dummies(X_train_raw, columns=categorical_cols)
X_test_enc = pd.get_dummies(X_test_raw, columns=categorical_cols)

# ALINEACIÓN DE COLUMNAS:
# El set de prueba puede tener menos servicios que el de entrenamiento o viceversa.
# se reindexa el test para que tenga EXACTAMENTE las mismas columnas que el train.
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

print(f"Dimensiones finales (Features): {X_train_enc.shape[1]}")

# Preprocesamiento: Binarización de Entradas (Input Binarization)
# La Neurona MP necesita 0s y 1s.
# Si valor > media de la columna en train -> 1, sino -> 0
print("Binarizando entradas (Threshold = Media)...")
train_means = X_train_enc.mean()

X_train_bin = (X_train_enc > train_means).astype(int).values
X_test_bin = (X_test_enc > train_means).astype(int).values

# Entrenar Neurona MP con NSL-KDD
# se usa un subset para entrenamiento rápido (la fuerza bruta es lenta con muchas columnas)
subset_size = 2000 
print(f"Entrenando con subset de {subset_size} muestras para eficiencia...")

mp_kdd = MPNeuron()
mp_kdd.fit(X_train_bin[:subset_size], y_train_kdd[:subset_size])

# Evaluación en el Test Set completo de KDD
print("Evaluando en KDD Test completo...")
y_pred_kdd = mp_kdd.predict(X_test_bin)
acc_kdd = accuracy_score(y_test_kdd, y_pred_kdd)

print(f"\nRESULTADO FINAL NSL-KDD:")
print(f"Accuracy de la Neurona MP: {acc_kdd:.4f}")


-PROCESAMIENTO Y USO DE NSL-KDD-
Datasets cargados correctamente.
Dimensiones finales (Features): 122
Binarizando entradas (Threshold = Media)...
Entrenando con subset de 2000 muestras para eficiencia...
Entrenando... Probando umbrales de 0 a 122
Mejor umbral encontrado: 9 con Accuracy: 0.7685
Evaluando en KDD Test completo...

RESULTADO FINAL NSL-KDD:
Accuracy de la Neurona MP: 0.7373
